In [ ]:
#@title
! wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local

! conda install -q -y -c rdkit rdkit 

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
!pip install git+https://github.com/keras-team/keras-tuner.git

#! python -m pip install "git+https://github.com/javadba/mpld3@display_fix"

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('./MyDrive')

In [ ]:
import numpy as np
import pandas as pd

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, DataStructs, Draw, Descriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools

In [ ]:
data = pd.read_csv('./MyDrive/My Drive/Colab Notebooks/data/delaney-processed.csv')[ [ 'Compound ID', 'smiles', 'measured log solubility in mols per litre' ] ]
# data.head()

In [ ]:
PandasTools.AddMoleculeColumnToFrame( data, 'smiles', 'Molecule' )
data = data.rename( columns={ 'measured log solubility in mols per litre':'solv' } )
#data.head()
#data.sort_values( by=['solv'], ascending=False ).head()
#data['solv'].max()
#high_sol = data.loc[ data['solv'] >= 1.5 ]
#high_sol

In [ ]:
def AromaticAtoms(m) :
    aa_atoms = [ m.GetAtomWithIdx(i).GetIsAromatic() for i in range( m.GetNumAtoms() ) ]
    aa_cnt = sum( 1 for a in aa_atoms if a == True )
    return aa_cnt / Descriptors.HeavyAtomCount(m)

data['mw'] = [ Descriptors.MolWt(mol) for mol in data[ 'Molecule' ] ]
data['MolLogP'] = [ Descriptors.MolLogP(mol) for mol in data[ 'Molecule' ] ]
data['NumRotatableBonds'] = [ Descriptors.NumRotatableBonds(mol) for mol in data[ 'Molecule' ] ]
data['AromP'] = [ AromaticAtoms(mol) for mol in data[ 'Molecule' ] ]
data.head()

In [ ]:
x = data[ [ 'MolLogP', 'mw', 'NumRotatableBonds', 'AromP' ] ]
y = data[ 'solv' ]
pd.concat( [x,y], axis=1 )

In [ ]:
from sklearn.model_selection import  train_test_split

X_train, X_test, Y_train, Y_test = train_test_split( x, y, test_size=0.3 )

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

model = linear_model.LinearRegression()
model.fit( X_train, Y_train )

Y_train_pred = model.predict( X_train )
Y_test_pred = model.predict( X_test )

In [ ]:
print( 'Coeff = ', model.coef_, ' Intercept = ', model.intercept_ )

print( 'MSE = %.2f' % mean_squared_error( Y_train, Y_train_pred ) )
print( 'R2 = %.2f' % r2_score( Y_train, Y_train_pred ) )

print( 'MSE = %.2f' % mean_squared_error( Y_test, Y_test_pred ) )
print( 'R2 = %.2f' % r2_score( Y_test, Y_test_pred ) )

print( 'LogS = %.3f + %.3f LogP + %.3f MW + %.3f RB + %.3f AP' % ( model.intercept_, model.coef_[0], model.coef_[1], model.coef_[2], model.coef_[3],) )
print( 'Delaney : LogS = 0.16 - 0.63 cLogP - 0.0062 MW + 0.066 RB - 0.74 AP' )

In [ ]:
import matplotlib.pyplot as plt
from rdkit.Chem.Draw import rdDepictor, rdMolDraw2D

In [ ]:
def moltosvg(mol,molSize=(225,75),kekulize=True):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    drawer.DrawMolecule(mc)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return svg.replace('svg:','')

In [ ]:
import matplotlib.pyplot as plt

plt.figure( figsize=(11,5) )

plt.subplot( 1, 2, 1 )
plt.scatter( x=Y_train, y=Y_train_pred, c="#7CAE00", alpha=0.3 )
z = np.polyfit( Y_train, Y_train_pred, 1 )
p = np.poly1d(z)
plt.plot( Y_train, p(Y_train), "#F8766D" )
plt.xlabel( 'Experimental LogS' )
plt.ylabel( 'Predicted LogS' )

plt.subplot( 1, 2, 2 )
plt.scatter( x=Y_test, y=Y_test_pred, c="#F8766D", alpha=0.3 )
z = np.polyfit( Y_test, Y_test_pred, 1 )
p = np.poly1d(z)
plt.plot( Y_test, p(Y_test), "#F8766D" )
plt.xlabel( 'Experimental LogS' )
plt.ylabel( 'Predicted LogS' )

plt.show()